In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import collections
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.set_printoptions(edgeitems=2)
torch.manual_seed(123)

In [2]:
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

In [3]:
from torchvision import datasets, transforms
data_path = 'E:/UNCC/UNCC ML/Project/data-unversioned/p1ch6/'
cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

Files already downloaded and verified


In [4]:
cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

Files already downloaded and verified


In [5]:
# label_map = {0: 0, 2: 1}
label_map = {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9}
#class_names = ['airplane', 'bird']
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

cifar2 = [(img, label_map[label])
          for img, label in cifar10
          if label in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]]
cifar2_val = [(img, label_map[label])
              for img, label in cifar10_val
              if label in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]]

In [6]:
class ResBlock(nn.Module):
    def __init__(self, n_chans):
        super(ResBlock, self).__init__()
        self.conv = nn.Conv2d(n_chans, n_chans, kernel_size=3,padding=1, bias=False)
        self.batch_norm = nn.BatchNorm2d(num_features=n_chans)
        torch.nn.init.kaiming_normal_(self.conv.weight,nonlinearity='relu')
        torch.nn.init.constant_(self.batch_norm.weight, 0.5)
        torch.nn.init.zeros_(self.batch_norm.bias)
    def forward(self, x):
        out = self.conv(x)
        out = self.batch_norm(out)
        out = torch.relu(out)
        return out + x

In [8]:
class NetResDeep(nn.Module):
    def __init__(self, n_chans1=32, n_blocks=10):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.resblocks = nn.Sequential(
        *(n_blocks * [ResBlock(n_chans=n_chans1)]))
        self.fc1 = nn.Linear(8 * 8 * n_chans1, 32)
        self.fc2 = nn.Linear(32, 10)
    def forward(self, x):
        out = F.max_pool2d(torch.relu(self.conv1(x)), 2)
        out = self.resblocks(out)
        out = F.max_pool2d(out, 2)
        out = out.view(-1, 8 * 8 * self.n_chans1)
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        return out

In [10]:
model = NetResDeep()
numel_list = [p.numel() for p in model.parameters()]
sum(numel_list), numel_list

(76074, [864, 32, 9216, 32, 32, 65536, 32, 320, 10])

In [12]:
img, _ = cifar2[0]
model = NetResDeep()
model(img.unsqueeze(0))

tensor([[-0.5090, -0.6696, -1.6149,  0.1984, -1.4875,  0.4024,  0.4739, -0.8115,
          2.0568, -1.1230]], grad_fn=<AddmmBackward0>)

In [13]:
device = (torch.device('cuda') if torch.cuda.is_available()
else torch.device('cpu'))
print(f"Training on device {device}.")

Training on device cuda.


In [14]:
import datetime

def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs + 1):
        epoch_start_time = time.time()
        loss_train = 0.0
        for imgs, labels in train_loader:
            imgs = imgs.to(device=device)
            labels = labels.to(device=device)
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            loss_train += loss.item()
        epoch_end_time = time.time()   
        if epoch == 1 or epoch % 20 == 0:
            print('{} Time Elapsed {}: Epoch {}, Training loss {}'.format(datetime.datetime.now(),(epoch_end_time - epoch_start_time), epoch, loss_train / len(train_loader)))

In [16]:
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=32,shuffle=True)

model = NetResDeep().to(device=device) #Instantiates our network with GPU ".to(device=device)"
optimizer = optim.SGD(model.parameters(), lr=1e-2) #the stochastic gradient descent optimizer we have been working with
loss_fn = nn.CrossEntropyLoss() #
training_loop(
    n_epochs = 200,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
    )

2024-12-09 22:44:59.747711 Time Elapsed 19.229727029800415: Epoch 1, Training loss 1.9328700008868256
2024-12-09 22:50:45.994027 Time Elapsed 18.551975965499878: Epoch 20, Training loss 0.5881304409991299
2024-12-09 22:56:48.965974 Time Elapsed 18.070319652557373: Epoch 40, Training loss 0.3869987768112126
2024-12-09 23:02:50.083288 Time Elapsed 17.98607897758484: Epoch 60, Training loss 0.2767000024267118
2024-12-09 23:08:51.923047 Time Elapsed 18.21383309364319: Epoch 80, Training loss 0.21373111720096202
2024-12-09 23:14:57.089576 Time Elapsed 18.779070377349854: Epoch 100, Training loss 0.18429653117670342
2024-12-09 23:21:07.128237 Time Elapsed 18.098594665527344: Epoch 120, Training loss 0.15046145342217707
2024-12-09 23:27:11.876552 Time Elapsed 19.196884393692017: Epoch 140, Training loss 0.11413382206550696
2024-12-09 23:33:12.737322 Time Elapsed 17.26853036880493: Epoch 160, Training loss 0.11061956615126375
2024-12-09 23:39:14.329448 Time Elapsed 17.78671097755432: Epoch 180

In [17]:
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=32, shuffle=False)

val_loader = torch.utils.data.DataLoader(cifar2_val, batch_size=32,shuffle=False)

def validate(model, train_loader, val_loader):
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0
        with torch.no_grad():
            for imgs, labels in loader:
                imgs = imgs.to(device=device)
                labels = labels.to(device=device)
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1)
                total += labels.shape[0]
                correct += int((predicted == labels).sum())
        print("Accuracy {}: {:.2f}".format(name , correct / total))
validate(model, train_loader, val_loader)

Accuracy train: 0.95
Accuracy val: 0.67
